In [271]:
import pandas as pd
import numpy as np

In [272]:
chennai_df = pd.read_csv('Chennai.csv')
chennai_df['dataset'] = 'Chennai'

hyderabad_df = pd.read_csv('Hyderabad.csv')
hyderabad_df['dataset'] = 'Hydebarad'

kolkata_df = pd.read_csv('Kolkata.csv')
kolkata_df['dataset'] = 'Kolkata'

In [273]:
df = pd.concat([chennai_df, hyderabad_df, kolkata_df])

In [274]:
df['property_type'] = df['property_type'].astype(str)

In [275]:
def drop_null_rows(columns):
    for column in columns:
        df.drop(df[df[column].isnull()].index.tolist(), axis=0, inplace=True)

In [276]:
df['bathroom'] = df['bathroom'].str.replace('bathrooms', '')

In [277]:
drop_null_rows(['seller_type', 'bathroom'])

In [278]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27068 entries, 0 to 9899
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   seller_type    27068 non-null  object 
 1   bedroom        27068 non-null  float64
 2   layout_type    27068 non-null  object 
 3   property_type  27068 non-null  object 
 4   locality       27068 non-null  object 
 5   price          27068 non-null  object 
 6   area           27068 non-null  float64
 7   furnish_type   27068 non-null  object 
 8   bathroom       27068 non-null  object 
 9   dataset        27068 non-null  object 
dtypes: float64(2), object(8)
memory usage: 2.3+ MB
